# Detect MGP

In [ ]:
# import the necessary packages
import numpy as np
import argparse
import glob
import os,sys
import cv2


# set up arrays for output
all_circles_as_array = []
filename = []
cx = []
cy = []
cr = []

# Extract circles from map images

## Define circle finder

In [ ]:
# define circles function
import cv2
import glob
import numpy as np
import os,sys

def find_circles(rmin, rmax, min_dist, p2):
    circles = cv2.HoughCircles(blur,cv2.HOUGH_GRADIENT,1,min_dist,
                                param1=p1,param2=p2,minRadius=rmin,maxRadius=rmax)
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            try:
                box = int(mult*r)
                box = int(mult*r)
                ROI = gray[y-box:y+box, x-box:x+box]
                ROI = cv2.resize(ROI, (64,64))
                all_circles_as_array.append(ROI)
                filename.append(imgname)
                cx.append(x)
                cy.append(y)
                cr.append(r)
            except:
                continue

## Set circle parameters

In [ ]:
################################## parameters ##################################
# global circles params
p1 = 20
# blur
b1 = 3
b2 = 3

# how much to increase bounding box around circle ROI
mult = 1.2

# define pass 1
min1 = 15
max1 = 40
dist1 = 20
p2_1 = 65

# define pass 2
min2 = 41
max2 = 70
dist2 = 40
p2_2 = 70

# define pass 3
min3 = 71
max3 = 130
dist3 = 70
p2_3 = 75

## Find circles

In [ ]:
# load filenames
# images are placed in folders within the input directory

inpath = 'input/***/*.jpg'
imgnames = sorted(glob.glob(inpath))

# exclude files that include 'ind', 'titl', or 'covr' in filenames, as well as index maps, which generally include '0000'
imgnames = [ x for x in imgnames if "ind" not in x and "titl" not in x and 'covr' not in x and 'cbd' not in x and '0000' not in x]

# load images
for imgname in imgnames:
    image = cv2.imread(imgname)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(b1,b2),0)

# find the circles!
    find_circles(min1, max1, dist1, p2_1)
    find_circles(min2, max2, dist2, p2_2)
    find_circles(min3, max3, dist3, p2_3)

# Construct the arrays
# X and filename arrays will be used for analysis below
X , filename = np.array(all_circles_as_array), np.array(filename)
print(":::::Circles found, input to array:::::")
print(("X array: ") + str(X.shape))

# Classify circular features

## Import CNN models

In [ ]:
# import CNN models
import tensorflow as tf


models = list()
for model in range(0,5):
    path = 'ML_models/ensemblemodel' + '-' + str(model)
    model = tf.keras.models.load_model(path)
    models.append(model)

## Import MLP models

In [ ]:
# import MLP models

import pickle as pkl
file = open('ML_models/MLP/mlp_models.pkl', 'rb')

pipes = pkl.load(file)

## Run classifiers

In [ ]:
# define thresholds and run models
CNNthresh = 0.2
MLPthresh = 0.9

# reshape X array to 2-dimensions for MLP
nsamples, nx, ny = X.shape
X_mlp = X.reshape((nsamples, nx*ny))

y_p_mlp = [pipe.predict_proba(X_mlp)[:,1] for pipe in pipes]
y_p_mlp = np.array(y_p_mlp)
y_p_mlp = np.mean(y_p_mlp, axis=0

y_p_cnn = [model.predict(X)[:,1] for model in models]
y_p_cnn = np.array(y_p_cnn)
y_p_cnn = np.mean(y_p_cnn, axis=0)

In [ ]:
# combine MLP and CNN predictions
y_p_all = (y_p_cnn*MLPthresh/CNNthresh + y_p_mlp*CNNthresh/MLPthresh)

# produce binary prediction
ALLthresh = 1.2
y_p_final = (y_p_all > ALLthresh).astype('int32')

In [ ]:
# export prediction results to CSV, including MLP, CNN, and combined prediction for each circular feature
# this allows you to select filenames according to your own thresholds for MLP, CNN, or combined predictions; or to use the binary threshold defined above
# you can use basic dataframe operations to produce a list of unique filenames, which you can then use to extract files from the original input data for manual inspection

import pandas as pd
dffile = pd.DataFrame(filenames)

dfy_p_all = pd.DataFrame(y_p_all)
dfy_p_final = pd.DataFrame(y_p_final)
dfy_p_CNN = pd.DataFrame(y_p_CNN)
dfy_p_MLP = pd.DataFrame(y_p_MLP)

df = pd.concat([dffile, dfy_p_all, dfy_p_final, dfy_p_CNN, dfy_p_MLP], axis=1)

df.columns = ['file', 'combined_prediction', 'binary_prediction', 'cnn_prediction', 'mlp_prediction']

df.to_csv (r'output.csv', index = False)